In [ ]:
import pandas as pd
import os
from bs4 import UnicodeDammit
import re
import numpy as np

In [ ]:
# Estableciendo directorio
tar = os.chdir(r"undisclosed")

# Leyendo base
df_contacts = pd.read_csv("final_versions/df_contacts_extended_genderized_arranged.csv")

In [ ]:
len(df_contacts)

### Arreglando teléfonos

In [ ]:
df_contacts.tel

In [ ]:
# Removiendo signo + en teléfonos

# define lambda function to remove '+' from 'tel' values that start with '+569'
def remove_plus(x):
    if isinstance(x, str) and x.startswith('+569'):
        return x[1:]
    else:
        return x

# apply lambda function to 'tel' column
df_contacts['tel'] = df_contacts['tel'].apply(lambda x: remove_plus(x))

print(df_contacts.tel)

In [ ]:
df_contacts.c1.head(20)

In [ ]:
df_contacts.c4.head(20)

In [ ]:
#Filling c1

df_contacts.loc[df_contacts['c1'].isna() & df_contacts['c4'].notna(), 'c1'] = df_contacts['c4']

df_contacts.c1.head(20)

In [ ]:
df_contacts['c1'] = df_contacts['c1'].replace('CEPES', 'CEPSA')

### Arreglando fecha de nacimiento

In [ ]:
df_contacts.fecha_nacimiento.isna().sum()

In [ ]:
# convert 'fecha_nacimiento' to datetime object and format as 'dd/mm/yyyy'
df_contacts['fecha_nacimiento_1'] = pd.to_datetime(df_contacts['fecha_nacimiento'], errors='coerce').dt.strftime('%d/%m/%Y')

In [ ]:
df_contacts.fecha_nacimiento.isna().sum()

### Categorizando ocupación: Profesión

In [ ]:
df_contacts['categoria_ocupacion'] = 0

In [ ]:
#Generalizable?
'''
def contains_economia(string):
    # create a regular expression pattern that matches whole words in the list, ignoring case and accents
    pattern = r"\b(" + "|".join([re.escape(s) for s in economia_finanzas]) + r")\b"
    return re.search(pattern, string, flags=re.IGNORECASE | re.UNICODE) is not None

def is_contained(df, list, searched_column, imputed_column, code):
    df[imputed_column] = df['profesion_procesada_corregida'].apply(lambda x: 4 if (type(x) == str and contains_public_institution(" " + x.lower() + " ")) else df_contacts['categoria_ocupacion'][df_contacts['profesion_procesada_corregida'] == x].iloc[0] if type(x) == str else x)
'''

In [ ]:
academia = [
    'docente',
    'profesor',
    'profesora',
    'investigador',
    'investigadora',
    'Investigador',
    'Profesor'
]

def contains_academia(string):
    # create a regular expression pattern that matches partial matches of the strings in the list, ignoring case and accents
    pattern = r"(^|\W)(" + "|".join([re.escape(s) for s in academia]) + r")(\W|$)"
    return re.search(pattern, str(string), flags=re.IGNORECASE | re.UNICODE) is not None

# apply the function to the 'institucion_procesada' column and assign 2 to 'categoria_ocupacion' if a match is found
df_contacts['categoria_ocupacion'] = df_contacts['profesion_procesada_corregida'].apply(lambda x: 2 if (type(x) == str and contains_academia(x)) else np.nan if pd.isna(x) else df_contacts['categoria_ocupacion'][df_contacts['profesion_procesada_corregida'] == x].iloc[0] if type(x) == str else x)

'''
def contains_academia(string):
    # create a regular expression pattern that matches whole words in the list, ignoring case and accents
    pattern = r"\b(" + "|".join([re.escape(s) for s in academia]) + r")\b"
    return re.search(pattern, string, flags=re.IGNORECASE | re.UNICODE) is not None

# apply the function to the 'institucion_procesada' column and assign 2 to 'categoria_ocupacion' if a match is found
df_contacts['categoria_ocupacion'] = df_contacts['profesion_procesada_corregida'].apply(lambda x: 2 if (type(x) == str and contains_academia(" " + x.lower() + " ")) else df_contacts['categoria_ocupacion'][df_contacts['profesion_procesada_corregida'] == x].iloc[0] if type(x) == str else x)
'''

In [ ]:
def update_category(row):
    if pd.isna(row["profesion_procesada_corregida"]) or row["profesion_procesada_corregida"] == "":
        return row["categoria_ocupacion"]
    elif "Profesor" in row["profesion_procesada_corregida"]:
        return 2
    else:
        return row["categoria_ocupacion"]

df_contacts["categoria_ocupacion"] = df_contacts.apply(update_category, axis=1)

In [ ]:
#df_prof = df_contacts.query('profesion_procesada_corregida == Profesor')

In [ ]:
df_contacts.profesion_procesada_corregida.iloc[2138]

In [ ]:
economia_finanzas = [
    'finanzas',
    'economista',
    'economía',
    'contador',
    'ingeniero comercial',
    'Contador'
]

def contains_economia(string):
    # create a regular expression pattern that matches whole words in the list, ignoring case and accents
    pattern = r"\b(" + "|".join([re.escape(s) for s in economia_finanzas]) + r")\b"
    return re.search(pattern, string, flags=re.IGNORECASE | re.UNICODE) is not None

# apply the function to the 'institucion_procesada' column and assign 2 to 'categoria_ocupacion' if a match is found
df_contacts['categoria_ocupacion'] = df_contacts['profesion_procesada_corregida'].apply(lambda x: 4 if (type(x) == str and contains_economia(" " + x.lower() + " ")) else df_contacts['categoria_ocupacion'][df_contacts['profesion_procesada_corregida'] == x].iloc[0] if type(x) == str else x)

In [ ]:
def update_category(row):
    if pd.isna(row["profesion_procesada_corregida"]) or row["profesion_procesada_corregida"] == "":
        return row["categoria_ocupacion"]
    elif "Ingeniero comercial" in row["profesion_procesada_corregida"]:
        return 4
    else:
        return row["categoria_ocupacion"]
    
df_contacts["categoria_ocupacion"] = df_contacts.apply(update_category, axis=1)

In [ ]:
ingenieria = [
    'ingeniero',
    'ingeniería',
    'ingeniera',
    'Ingeniero'
]

def contains_ingenieria(string):
    # create a regular expression pattern that matches whole words in the list, ignoring case and accents
    pattern = r"\b(" + "|".join([re.escape(s) for s in ingenieria]) + r")\b"
    return re.search(pattern, string, flags=re.IGNORECASE | re.UNICODE) is not None

# apply the function to the 'institucion_procesada' column and assign 2 to 'categoria_ocupacion' if a match is found
df_contacts['categoria_ocupacion'] = df_contacts['profesion_procesada_corregida'].apply(lambda x: 5 if (type(x) == str and contains_ingenieria(" " + x.lower() + " ")) else df_contacts['categoria_ocupacion'][df_contacts['profesion_procesada_corregida'] == x].iloc[0] if type(x) == str else x)

In [ ]:
empresarios_ejecutivos = [
    'gerente',
    'presidente',
    'director',
    'directora',
    'empresario',
    'empresaria'
]

def contains_empresarios(string):
    # create a regular expression pattern that matches whole words in the list, ignoring case and accents
    pattern = r"\b(" + "|".join([re.escape(s) for s in empresarios_ejecutivos]) + r")\b"
    return re.search(pattern, string, flags=re.IGNORECASE | re.UNICODE) is not None

# apply the function to the 'institucion_procesada' column and assign 2 to 'categoria_ocupacion' if a match is found
df_contacts['categoria_ocupacion'] = df_contacts['profesion_procesada_corregida'].apply(lambda x: 6 if (type(x) == str and contains_empresarios(" " + x.lower() + " ")) else df_contacts['categoria_ocupacion'][df_contacts['profesion_procesada_corregida'] == x].iloc[0] if type(x) == str else x)

In [ ]:
salud = [
    'médico',
    'médica',
    'enfermera',
    'enfermero',
    'directora',
    'psicóloga',
    'psicólogo',
    'Enfermero'
]

def contains_salud(string):
    # create a regular expression pattern that matches whole words in the list, ignoring case and accents
    pattern = r"\b(" + "|".join([re.escape(s) for s in salud]) + r")\b"
    return re.search(pattern, string, flags=re.IGNORECASE | re.UNICODE) is not None

# apply the function to the 'institucion_procesada' column and assign 2 to 'categoria_ocupacion' if a match is found
df_contacts['categoria_ocupacion'] = df_contacts['profesion_procesada_corregida'].apply(lambda x: 7 if (type(x) == str and contains_salud(" " + x.lower() + " ")) else df_contacts['categoria_ocupacion'][df_contacts['profesion_procesada_corregida'] == x].iloc[0] if type(x) == str else x)

In [ ]:
sociologia_ciencias_politicas = [
    'cientista político',
    'cientista política',
    'politólogo',
    'politóloga',
    'sociólogo',
    'socióloga',
    'sociología'
]

def contains_sociologia(string):
    # create a regular expression pattern that matches whole words in the list, ignoring case and accents
    pattern = r"\b(" + "|".join([re.escape(s) for s in sociologia_ciencias_politicas]) + r")\b"
    return re.search(pattern, string, flags=re.IGNORECASE | re.UNICODE) is not None

# apply the function to the 'institucion_procesada' column and assign 2 to 'categoria_ocupacion' if a match is found
df_contacts['categoria_ocupacion'] = df_contacts['profesion_procesada_corregida'].apply(lambda x: 8 if (type(x) == str and contains_sociologia(" " + x.lower() + " ")) else df_contacts['categoria_ocupacion'][df_contacts['profesion_procesada_corregida'] == x].iloc[0] if type(x) == str else x)

In [ ]:
abogados = [
    'abogado',
    'abogada',
    'derecho',
    'abogados'
]

def contains_abogado(string):
    # create a regular expression pattern that matches whole words in the list, ignoring case and accents
    pattern = r"\b(" + "|".join([re.escape(s) for s in abogados]) + r")\b"
    return re.search(pattern, string, flags=re.IGNORECASE | re.UNICODE) is not None

# apply the function to the 'institucion_procesada' column and assign 2 to 'categoria_ocupacion' if a match is found
df_contacts['categoria_ocupacion'] = df_contacts['profesion_procesada_corregida'].apply(lambda x: 9 if (type(x) == str and contains_abogado(" " + x.lower() + " ")) else df_contacts['categoria_ocupacion'][df_contacts['profesion_procesada_corregida'] == x].iloc[0] if type(x) == str else x)

In [ ]:
agronomia_forestales = [
    'agronomo',
    'forestal',
    'Agrónomo'
]

def contains_agronomo(string):
    # create a regular expression pattern that matches whole words in the list, ignoring case and accents
    pattern = r"\b(" + "|".join([re.escape(s) for s in agronomia_forestales]) + r")\b"
    return re.search(pattern, string, flags=re.IGNORECASE | re.UNICODE) is not None

# apply the function to the 'institucion_procesada' column and assign 2 to 'categoria_ocupacion' if a match is found
df_contacts['categoria_ocupacion'] = df_contacts['profesion_procesada_corregida'].apply(lambda x: 10 if (type(x) == str and contains_agronomo(" " + x.lower() + " ")) else df_contacts['categoria_ocupacion'][df_contacts['profesion_procesada_corregida'] == x].iloc[0] if type(x) == str else x)

### Categorizando ocupación: Institución

In [ ]:
instituciones_publicas = [
    "BancoEstado", "BE",
    "Caja de Previsión de la Defensa Nacional", "CAPREDENA",
    "Caja de Previsión de la Policía de Investigaciones de Chile", "DIPRECA",
    "Caja de Previsión de la Seguridad Social", "IPS",
    "Caja de Previsión de los Carabineros de Chile", "CARE",
    "Caja Nacional de Empleados Públicos y Periodistas", "CNP",
    "Comisión Nacional de Investigación Científica y Tecnológica", "CONICYT",
    "Consejo Nacional de la Cultura y las Artes", "CNCA",
    "Contraloría General de la República", "CGR",
    "Corporación de Fomento de la Producción", "CORFO",
    "Corporación Nacional del Cobre de Chile", "Codelco",
    "Corporación Nacional Forestal", "CONAF",
    "Dirección de Aeropuertos", "DAP",
    "Dirección de Bibliotecas, Archivos y Museos", "DIBAM",
    "Dirección de Compras y Contratación Pública", "CHILECOMPRA",
    "Dirección de Obras Hidráulicas", "DOH",
    "Dirección General de Aeronáutica Civil", "DGAC",
    "Dirección General de Aguas", "DGA",
    "Dirección General de Movilización Nacional", "DGMN",
    "Dirección General de Obras Públicas", "DGOP",
    "Dirección General del Crédito Prendario", "DICREP",
    "Dirección General del Territorio Marítimo y de Marina Mercante", "DIRECTEMAR",
    "Dirección Nacional de Fronteras y Límites del Estado", "DIFROL",
    "Empresa de Ferrocarriles del Estado", "EFE",
    "Empresa Nacional del Petróleo", "ENAP",
    "Fondo de Solidaridad e Inversión Social", "FOSIS",
    "Fondo Nacional de Salud", "FONASA",
    "Fondo Nacional de Desarrollo Regional", "FNDR",
    "Fondo Nacional de Seguridad Pública", "FNSP",
    "Instituto de Desarrollo Agropecuario", "INDAP",
    "Junta Nacional de Jardines Infantiles", "JUNJI",
    "Ministerio de Agricultura", "MINAGRI",
    "Ministerio de Bienes Nacionales", "MINBAS",
    "Ministerio de Desarrollo Social y Familia", "MDS",
    "Ministerio de Educación", "MINEDUC",
    "Ministerio de Energía", "MINENERGIA",
    "Ministerio de Hacienda", "MINHACIENDA",
    "Ministerio de Justicia y Derechos Humanos", "MINJUSTICIA",
    "Ministerio de Obras Públicas", "MOP",
    "Ministerio de Relaciones Exteriores", "MINREL",
    "Ministerio de Salud", "MINSAL",
    "Ministerio de Transportes y Telecomunicaciones", "MTT",
    "Municipalidad", "MUNI",
    "Oficina Nacional de Emergencia", "ONEMI",
    "Programa de las Naciones Unidas para el Desarrollo", "PNUD",
    "Registro Civil e Identificación", "RCI",
    "Servicio Agrícola y Ganadero", "SAG",
    "Servicio Electoral", "SERVEL",
    "Servicio Hidrográfico y Oceanográfico de la Armada", "SHOA",
    "Servicio Médico Legal", "SML",
    "Servicio Nacional de Aduanas", "ADUANAS",
    "Servicio Nacional de Capacitación y Empleo", "SENCE",
    "Servicio Nacional de Geología y Minería", "SERNAGEOMIN",
    "Servicio Nacional de la Discapacidad", "SENADIS",
    "Servicio Nacional de la Mujer y la Equidad de Género", "SERNAMEG",
    "Servicio Nacional del Adulto Mayor", "SENAMA",
    "Servicio Nacional del Consumidor", "SERNAC",
    "Servicio Nacional del Patrimonio Cultural", "SNPC",
    "Servicio Nacional Forestal y de Fauna Silvestre", "CONAF",
    "Servicio Nacional de Pesca y Acuicultura", "SERNAPESCA",
    "Servicio Nacional para la Prevención y Rehabilitación del Consumo de Drogas y Alcohol", "SENDA",
    "Servicio Nacional de Turismo", "SERNATUR",
    "Servicio Nacional de la Discapacidad", "SENADIS",
    "Servicio Nacional de la Mujer y la Equidad de Género", "SERNAMEG",
    "Superintendencia de Bancos e Instituciones Financieras", "SBIF",
    "Superintendencia de Electricidad y Combustibles", "SEC",
    "Superintendencia de Insolvencia y Reemprendimiento", "SUPERIR",
    "Superintendencia de Pensiones", "SP",
    "Superintendencia de Salud", "SIS",
    "Superintendencia de Seguridad Social", "SUSESO",
    "Superintendencia de Servicios Sanitarios", "SISS",
    "Superintendencia del Medio Ambiente", "SMA",
    "Subsecretaría de Economía y Empresas de Menor Tamaño", "SUBEM",
    "Subsecretaría de Educación", "SUBEDUC",
    "Subsecretaría de Energía", "SUBENERGIA",
    "Subsecretaría de Hacienda", "SUBHACIENDA",
    "Subsecretaría de Obras Públicas", "SUBOP",
    "Subsecretaría de Pesca y Acuicultura", "SUBPESCA",
    "Subsecretaría de Prevención del Delito", "SUBSEGPUB",
    "Subsecretaría de Previsión Social", "SUBPREV",
    "Subsecretaría de Relaciones Exteriores", "SUBREX",
    "Subsecretaría de Telecomunicaciones", "SUBTEL",
    "Subsecretaría de Transportes", "SUBTRANS",
    "Subsecretaría General de Gobierno", "SUBGOB",
    "Tribunal Constitucional", "TC",
    "Tribunal de Defensa de la Libre Competencia", "TDLC",
    "Tribunal Electoral Regional", "TER",
    "Tribunal Oral en lo Penal", "TOP",
    "Carabineros",
    "Ejercito",
    "Armada",
    "Fuerza aérea",
    "Embajada",
    "Seremi"
]

In [ ]:
instituciones_educacion_superior = [
    'Universidad',
    'Universidad Academia de Humanismo Cristiano', 'UAHC', 
    'Universidad Adolfo Ibáñez', 'UAI', 
    'Universidad Adventista de Chile', 'UAC', 
    'Universidad Alberto Hurtado', 'UAH', 
    'Universidad Andrés Bello', 'UNAB', 'Universidad andres bello'
    'Universidad Arturo Prat', 'UNAP', 
    'Universidad Austral de Chile', 'UACh', 
    'Universidad Autónoma de Chile', 'UA', 'Universidad A'
    'Universidad Bernardo O’Higgins', 'UBO', 
    'Universidad Bolivariana', 'UB', 
    'Universidad Católica Cardenal Raúl Silva Henríquez', 'UCSH', 
    'Universidad Católica de la Santísima Concepción', 'UCSC', 
    'Universidad Católica de la VI Región', 'UCV', 
    'Universidad Católica del Maule', 'UCM', 
    'Universidad Católica del Norte', 'UCN', 
    'Pontificia Universidad Católica de Chile', 'PUC', 'UC', 'Universidad Católica',
    'PUCV',
    'Universidad Central de Chile', 'UCEN', 'Universidad Central',
    'Universidad Chileno-Británica de Cultura', 'UCBC', 
    'Universidad de Aconcagua', 'UDA', 
    'Universidad de Antofagasta', 'UA', 
    'Universidad de Artes, Ciencias y Comunicación', 'UNIACC', 
    'Universidad de Atacama', 'UDA', 
    'Universidad de Chile', 'UCH', 
    'Universidad de Ciencias de la Informática', 'UCINF', 
    'Universidad de Concepción', 'UdeC', 
    'Universidad de Concepción del Uruguay', 'UCU', 
    'Universidad de Los Andes', 'UANDES', 
    'Universidad de Los Lagos', 'ULAGOS', 
    'Universidad de Magallanes', 'UMAG', 
    'Universidad de Playa Ancha de Ciencias de la Educación', 'UPLA', 
    'Universidad de Santiago de Chile', 'USACH', 
    'Universidad de Talca', 'UTAL', 
    'Universidad de Tarapacá', 'UTA', 
    'Universidad de Valparaíso', 'UV', 
    'Universidad del Bío-Bío', 'UBB', 
    'Universidad del Desarrollo', 'UDD', 
    'Universidad del Pacífico', 'UDP', 
    'Universidad Diego Portales', 'UDP', 
    'Universidad Finis Terrae', 'UFT', 
    'Universidad Gabriela Mistral', 'UGM', 
    'Universidad Iberoamericana de Ciencias y Tecnología', 'UICT', 
    'Universidad La República', 'ULARE', 
    'Universidad Mayor', 'UMAYOR', 
    'Universidad Metropolitana de Ciencias de la Educación', 'UMCE', 
    'Universidad Miguel de Cervantes', 'UMC', 
    'Universidad Nacional Andrés Bello', 'UNAB',
    'Universidad Tecnológica de Chile Inacap', 'INACAP', 
    'Universidad Técnica Federico Santa María', 'USM', 
    'Universidad Viña del Mar', 'UVM',
    'AIEP',     
    'CFT de Tarapacá',     
    'CFT de Valparaíso',     
    'CFT de la Región de Coquimbo',     
    'CFT de la Región del Libertador General Bernardo O’Higgins', 
    'CFT Lota-Arauco',     
    'CFT San Agustín',     
    'CFT Santo Tomás',     
    'CFT Teodoro Wickel',     
    'CFT UV',     
    'CFT de Los Lagos',     
    'CFT de la Región de Aysén',     
    'CFT de la Región de Magallanes y Antártica Chilena',     
    'CFT estatal de La Araucanía',     
    'CFT estatal de Los Ríos',     
    'CFT estatal de O’Higgins',     
    'CFT estatal de Tarapacá',     
    'CFT estatal de la Región de Valparaíso',     
    'CFT estatal de la Región de Coquimbo',     
    'CFT estatal de la Región del Maule',     
    'CFT estatal de la Región de Los Lagos',     
    'CFT estatal de la Región de Aysén',     
    'CFT estatal de la Región de Magallanes y de la Antártica Chilena',     
    'CFT de la Universidad de Santiago de Chile', 'CFT USACH',     
    'CFT de la Universidad Técnica Federico Santa María', 
    'CFT USM'
]

In [ ]:
mineras = [
    'minera',
    'mineria',
    'minero'
    'minerals', 'antofagasta minerals',
    'bhp',
    'anglo american'
]

In [ ]:
'''
def contains_superior_education_institution(string):
    # create a regular expression pattern that matches whole words in the list, ignoring case and accents
    pattern = r"\b(" + "|".join([re.escape(s) for s in instituciones_educacion_superior]) + r")\b"
    return re.search(pattern, string, flags=re.IGNORECASE | re.UNICODE) is not None

# apply the function to the 'institucion_procesada' column and assign 2 to 'categoria_ocupacion' if a match is found
df_contacts['categoria_ocupacion'] = df_contacts['institucion_procesada'].apply(lambda x: 2 if (type(x) == str and contains_superior_education_institution(" " + x.lower() + " ")) else df_contacts['categoria_ocupacion'][df_contacts['institucion_procesada'] == x].iloc[0] if type(x) == str else x)
'''

In [ ]:
df_contacts.categoria_ocupacion.value_counts()

In [ ]:
def contains_public_institution(string):
    # create a regular expression pattern that matches whole words in the list, ignoring case and accents
    pattern = r"\b(" + "|".join([re.escape(s) for s in instituciones_publicas]) + r")\b"
    return re.search(pattern, string, flags=re.IGNORECASE | re.UNICODE) is not None

# apply the function to the 'institucion_procesada' column and assign 2 to 'categoria_ocupacion' if a match is found
df_contacts['categoria_ocupacion'] = df_contacts['institucion_procesada'].apply(lambda x: 1 if (type(x) == str and contains_public_institution(" " + x.lower() + " ")) else df_contacts['categoria_ocupacion'][df_contacts['institucion_procesada'] == x].iloc[0] if type(x) == str else x)

In [ ]:
df_contacts.categoria_ocupacion.value_counts()

In [ ]:
def contains_minera(string):
    # create a regular expression pattern that matches whole words in the list, ignoring case and accents
    pattern = r"\b(" + "|".join([re.escape(s) for s in mineras]) + r")\b"
    return re.search(pattern, string, flags=re.IGNORECASE | re.UNICODE) is not None

# apply the function to the 'institucion_procesada' column and assign 2 to 'categoria_ocupacion' if a match is found
df_contacts['categoria_ocupacion'] = df_contacts['institucion_procesada'].apply(lambda x: 3 if (type(x) == str and contains_minera(" " + x.lower() + " ")) else df_contacts['categoria_ocupacion'][df_contacts['institucion_procesada'] == x].iloc[0] if type(x) == str else x)

In [ ]:
def update_category(row):
    if "@bhpbilliton.com" in row["email"]:
        return 3
    else:
        return row["categoria_ocupacion"]

df_contacts["categoria_ocupacion"] = df_contacts.apply(update_category, axis=1)

In [ ]:
df_contacts.categoria_ocupacion.value_counts()

In [ ]:
len(df_contacts)

### Categorizando ocupación: Estudiantes

In [ ]:
estudiantes = [
    'estudiante',
    'alumno'
]

def contains_estudiante(string):
    # create a regular expression pattern that matches whole words in the list, ignoring case and accents
    pattern = r"\b(" + "|".join([re.escape(s) for s in estudiantes]) + r")\b"
    return re.search(pattern, string, flags=re.IGNORECASE | re.UNICODE) is not None

# apply the function to the 'institucion_procesada' column and assign 2 to 'categoria_ocupacion' if a match is found
df_contacts['categoria_ocupacion'] = df_contacts['profesion_procesada_corregida'].apply(lambda x: 11 if (type(x) == str and contains_estudiante(" " + x.lower() + " ")) else df_contacts['categoria_ocupacion'][df_contacts['profesion_procesada_corregida'] == x].iloc[0] if type(x) == str else x)

### Last try Ing. Comercial, Profesor y Cientista Político

In [ ]:
def update_category_prof(row):
    if pd.isna(row["profesion_procesada_corregida"]) or row["profesion_procesada_corregida"] == "":
        return row["categoria_ocupacion"]
    elif "Profesor" in row["profesion_procesada_corregida"]:
        return 2
    else:
        return row["categoria_ocupacion"]

df_contacts["categoria_ocupacion"] = df_contacts.apply(update_category_prof, axis=1)

In [ ]:
def update_category_ingeco(row):
    if pd.isna(row["profesion_procesada_corregida"]) or row["profesion_procesada_corregida"] == "":
        return row["categoria_ocupacion"]
    elif "Ingeniero comercial" in row["profesion_procesada_corregida"]:
        return 4
    else:
        return row["categoria_ocupacion"]

df_contacts["categoria_ocupacion"] = df_contacts.apply(update_category_ingeco, axis=1)

In [ ]:
def update_category_ingeco(row):
    if pd.isna(row["profesion_procesada_corregida"]) or row["profesion_procesada_corregida"] == "":
        return row["categoria_ocupacion"]
    elif "Cientista político" in row["profesion_procesada_corregida"]:
        return 8
    else:
        return row["categoria_ocupacion"]

df_contacts["categoria_ocupacion"] = df_contacts.apply(update_category_ingeco, axis=1)

In [ ]:
df_contacts['categoria_ocupacion'] = df_contacts['categoria_ocupacion'].fillna(0)

In [ ]:
#df_contacts.to_excel('final_versions/df_contacts_final_testing1004.xlsx')

In [ ]:
df_contacts.categoria_ocupacion.value_counts()

In [ ]:
df_contacts.head()

### Cambiando valores de género

In [ ]:
# replace values in the 'genero' column
df_contacts['genero'] = df_contacts['genero'].replace({'M': 'Masculino',
                                                       'F': 'Femenino',
                                                       None: 'N/E',
                                                       '': 'N/E'})

### Limpiando columnas y arreglando nombres

In [ ]:
cols_keep_complete = [
    'email',
    'nombre',
    'apellido',
    'institucion',
    'profesion',
    'tel',
    'c1',
    'c2',
    'c3',
    'genero',
    'member_rating',
    'cc',
    'region',
    'institucion_procesada',
    'profesion_procesada_corregida',
    'new_latitude',
    'new_longitude',
    'address',
    'comuna',
    'fecha_nacimiento_1',
    'categoria_ocupacion'
]

In [ ]:
df_contacts_complete = df_contacts[cols_keep_complete]

In [ ]:
df_contacts_complete = df_contacts_complete.rename(columns = {
    'email': 'Email', 
    'nombre': 'Nombre', 
    'apellido': 'Apellido',
    'fecha_nacimiento_1': 'Fecha de Nacimiento',
    'categoria_ocupacion': 'Área de Ocupación',
    'tel': 'Teléfono',
    'c1': 'Uso Interno',
    'genero': 'Género',
    'comuna': 'Comuna',
    'new_latitude': 'Latitud',
    'new_longitude': 'Longitud',
    'Address': 'Dirección',
    'institucion': 'Institución',
    'institucion_procesada': 'Institución Procesada',
    'profesion': 'Profesión',
    'profesión_procesada_corregida': 'Profesión Procesada'
    }
)

In [ ]:
df_contacts_complete = df_contacts_complete.reindex(columns = [
    'Email',
    'Nombre',
    'Apellido',
    'Fecha de Nacimiento',
    'Área de Ocupación',
    'Teléfono',
    'Uso Interno',
    'Género',
    'Comuna',
    'Dirección',
    'Latitud',
    'Longitud',
    'Institución',
    'Institución Procesada',
    'Profesión',
    'Profesión Procesada',
    'cc',
    'region',
    'c2',
    'c3',
    'member_rating'
])

In [ ]:
df_contacts_complete = df_contacts_complete.reset_index(drop=True)

In [ ]:
cols_keep_simplified = [
    'Email',
    'Nombre',
    'Apellido',
    'Fecha de Nacimiento',
    'Área de Ocupación',
    'Teléfono',
    'Uso Interno',
    'Género',
    'Comuna',
]

In [ ]:
df_contacts_simplified = df_contacts_complete[cols_keep_simplified]
df_contacts_simplified = df_contacts_simplified.reset_index(drop=True)

In [ ]:
df_contacts_complete.to_excel('final_versions/VF/CEP_contactos_completo.xlsx')
df_contacts_simplified.to_excel('final_versions/VF/CEP_contactos_simplificado.xlsx')